# 3304. Find the K-th Character in String Game I

# Easy

> Alice and Bob are playing a game. Initially, Alice has a string word = "a".

You are given a positive integer k.

Now Bob will ask Alice to perform the following operation forever:

Generate a new string by changing each character in word to its next character in the English alphabet, and append it to the original word.
For example, performing the operation on "c" generates "cd" and performing the operation on "zb" generates "zbac".

Return the value of the kth character in word, after enough operations have been done for word to have at least k characters.

Note that the character 'z' can be changed to 'a' in the operation.

# Example 1:

```
Input: k = 5

Output: "b"

Explanation:

Initially, word = "a". We need to do the operation three times:

Generated string is "b", word becomes "ab".
Generated string is "bc", word becomes "abbc".
Generated string is "bccd", word becomes "abbcbccd".
```

# Example 2:

```
Input: k = 10

Output: "c"

```

# Constraints:

- 1 <= k <= 500


For the problem "Find the K-th Character in String Game I", we need to determine the character at a specific index `k` after a series of string transformations. The string starts with "a", and in each step, a new string is generated by shifting each character of the current `word` to its next character in the alphabet (with 'z' wrapping to 'a'), and this new string is appended to the `word`.

Let's analyze the string growth:

- **Initial:** `word = "a"`, length = 1
- **1st operation:**
  - `word` is "a".
  - Shifted "a" is "b".
  - `word` becomes "ab". Length = 2.
- **2nd operation:**
  - `word` is "ab".
  - Shifted "a" is "b", shifted "b" is "c".
  - Shifted string is "bc".
  - `word` becomes "abbc". Length = 4.
- **3rd operation:**
  - `word` is "abbc".
  - Shifted "a" is "b", shifted "b" is "c", shifted "b" is "c", shifted "c" is "d".
  - Shifted string is "bccd".
  - `word` becomes "abbcbccd". Length = 8.

Notice a pattern: The length of the string doubles with each operation. The length after `n` operations will be $2^n$. Since $k$ is at most 500, we don't need to simulate too many operations. $2^8 = 256$, $2^9 = 512$. So, for $k \\le 500$, at most 9 operations will be sufficient to have at least `k` characters.

Here are a few approaches to solve this problem:

### Approach 1: Simulation (Direct String Manipulation)

This is the most straightforward approach. We can directly simulate the process until the `word` string has at least `k` characters.

```python
def findKthCharacter_simulation(k: int) -> str:
    word = "a"

    while len(word) < k:
        new_part = ""
        for char in word:
            if char == 'z':
                new_part += 'a'
            else:
                new_part += chr(ord(char) + 1)
        word += new_part

    return word[k - 1]

# Example usage:
# print(findKthCharacter_simulation(5))  # Output: b
# print(findKthCharacter_simulation(10)) # Output: c
```

**Complexity Analysis:**

- **Time Complexity:** In each step, the string length doubles. Let `L` be the final length of the string (which is the smallest power of 2 greater than or equal to `k`). The number of operations will be approximately $\\log\_2 k$. In each operation, we iterate through the current string to build the new part, which takes `O(current_length)` time. So, the total time complexity will be $O(L)$, which is roughly $O(k)$. Given $k \\le 500$, $L \\le 512$, this approach is very efficient.
- **Space Complexity:** We are storing the `word` string, which can grow up to `L` characters. So, the space complexity is $O(L)$, which is roughly $O(k)$.

### Approach 2: Recursive Observation (Pattern Recognition)

Let's analyze the structure of the string `word`.
`S_0 = "a"`
`S_1 = S_0 + shift(S_0) = "a" + "b" = "ab"`
`S_2 = S_1 + shift(S_1) = "ab" + "bc" = "abbc"`
`S_3 = S_2 + shift(S_2) = "abbc" + "bccd" = "abbcbccd"`

We can observe that `S_n` has length $2^n$.
If `k` is in the first half of `S_n` (i.e., `k <= len(S_{n-1})`), then the `k`-th character of `S_n` is simply the `k`-th character of `S_{n-1}`.
If `k` is in the second half of `S_n` (i.e., `k > len(S_{n-1})`), then the `k`-th character of `S_n` is the `(k - len(S_{n-1}))`-th character of `shift(S_{n-1})`. This means it's the next character in the alphabet of the `(k - len(S_{n-1}))`-th character of `S_{n-1}`.

This gives us a recursive structure. We need to find the `k`-th character in `S_n`.

Let `find_char(current_k, current_length, start_char_code)` be a function that returns the character.
`current_length` will always be a power of 2.

```python
def findKthCharacter_recursive(k: int) -> str:
    # Adjust k to be 0-indexed
    k -= 1

    current_length = 1
    # Find the smallest length (power of 2) that is >= k + 1
    while current_length <= k:
        current_length *= 2

    # We will trace back to the initial 'a'
    # The 'start_char_code' represents the ASCII value of 'a'
    # 'offset' is how many times the character has been shifted

    offset = 0 # How many times the character at index k has been shifted

    while current_length > 1:
        half_length = current_length // 2

        if k < half_length:
            # The character is in the first half, so it's from the previous iteration's 'word'
            pass # k remains the same, offset remains the same
        else:
            # The character is in the second half, so it's from the shifted part
            k -= half_length # Adjust k to be relative to the start of the shifted part
            offset += 1 # The character has been shifted one more time

        current_length = half_length

    # At this point, current_length is 1, and k is 0.
    # The character at index 0 in the original "a" has been shifted 'offset' times.

    initial_char_code = ord('a')
    final_char_code = (initial_char_code - ord('a') + offset) % 26 + ord('a')

    return chr(final_char_code)

# Example usage:
# print(findKthCharacter_recursive(5))  # Output: b
# print(findKthCharacter_recursive(10)) # Output: c
```

**Complexity Analysis:**

- **Time Complexity:** In each step of the `while` loop, `current_length` is halved. This takes `O(log k)` iterations. All operations inside the loop are constant time. So, the total time complexity is `O(log k)`.
- **Space Complexity:** `O(1)` as we are only using a few variables.

### Approach 3: Bit Manipulation (Optimized Recursion)

The recursive approach can be thought of in terms of binary representations of `k`. The `k` value and its relation to powers of 2 directly tells us whether we are in the original part or the shifted part.

If we represent `k-1` (0-indexed) in binary:
Let `k_idx = k - 1`.
The number of shifts `offset` can be determined by how many times we "cross" the midpoint of the string.
Each bit in the binary representation of `k_idx` can be related to this.

Consider `k = 5`. `k_idx = 4`. Binary `100`.
Length `1 -> 2 -> 4 -> 8`.
`current_length = 8`.
`k_idx = 4`.
`half_length = 4`.
`k_idx >= half_length` (4 \>= 4) is true.
`k_idx = 4 - 4 = 0`.
`offset = 1`.
`current_length = 4`.
`half_length = 2`.
`k_idx < half_length` (0 \< 2) is true.
`k_idx` remains 0.
`offset` remains 1.
`current_length = 2`.
`half_length = 1`.
`k_idx < half_length` (0 \< 1) is true.
`k_idx` remains 0.
`offset` remains 1.
`current_length = 1`. Loop ends.
Result: `chr((ord('a') - ord('a') + 1) % 26 + ord('a')) = chr(ord('a') + 1) = 'b'`. This matches Example 1.

Consider `k = 10`. `k_idx = 9`. Binary `1001`.
Length `1 -> 2 -> 4 -> 8 -> 16`.
`current_length = 16`.
`k_idx = 9`.
`half_length = 8`.
`k_idx >= half_length` (9 \>= 8) is true.
`k_idx = 9 - 8 = 1`.
`offset = 1`.
`current_length = 8`.
`half_length = 4`.
`k_idx < half_length` (1 \< 4) is true.
`k_idx` remains 1.
`offset` remains 1.
`current_length = 4`.
`half_length = 2`.
`k_idx < half_length` (1 \< 2) is true.
`k_idx` remains 1.
`offset` remains 1.
`current_length = 2`.
`half_length = 1`.
`k_idx >= half_length` (1 \>= 1) is true.
`k_idx = 1 - 1 = 0`.
`offset = 2`.
`current_length = 1`. Loop ends.
Result: `chr((ord('a') - ord('a') + 2) % 26 + ord('a')) = chr(ord('a') + 2) = 'c'`. This matches Example 2.

This demonstrates that the recursive approach is effectively navigating the binary representation of `k - 1`. The `offset` counts how many '1's are encountered when `k_idx` is repeatedly adjusted by subtracting `half_length`.

### Which approach to choose?

- For small `k` (like `k <= 500`), **Approach 1 (Simulation)** is perfectly fine and often easiest to understand and implement correctly. Its $O(k)$ time complexity is very efficient for $k=500$.
- For larger `k` (e.g., $k$ up to $10^{18}$), **Approach 2 (Recursive Observation)** or the logic described in **Approach 3 (Bit Manipulation)** would be necessary due to their `O(log k)` time complexity and `O(1)` space complexity. The problem constraint `1 <= k <= 500` makes the simulation a viable and simple solution.

Given the constraints, I would recommend the **Simulation (Approach 1)** for its simplicity and directness. However, if asked for all possible approaches, understanding the recursive/bit manipulation method is valuable.

Let's provide the code for the recommended simulation approach.

```python
class Solution:
    def findKthCharacter(self, k: int) -> str:
        word = "a"

        # We need to ensure the word has at least k characters.
        # The length of the string doubles in each operation.
        # For k <= 500, the maximum length will be 512 (2^9).
        # So, at most 9 operations are needed.
        while len(word) < k:
            new_part = ""
            for char in word:
                # Shift character to the next in alphabet, 'z' wraps to 'a'
                if char == 'z':
                    new_part += 'a'
                else:
                    new_part += chr(ord(char) + 1)
            word += new_part

        # Return the k-th character (1-indexed)
        return word[k - 1]

```


To solve the "Find the K-th Character in String Game I" problem using an Object-Oriented Programming (OOP) approach in Python, we can encapsulate the game's logic within a class. This class will manage the `word` and provide a method to find the `k`-th character.

We'll primarily focus on two OOP approaches:

1.  **OOP with Direct Simulation:** This approach directly implements the string generation process within the class. It's simple and efficient enough for the given constraints.
2.  **OOP with Recursive Logic (Optimized):** This approach encapsulates the recursive pattern recognition logic. While overkill for `k <= 500`, it demonstrates a more scalable OOP design for larger `k` values.

### OOP Approach 1: Direct Simulation

This approach creates a class that simulates the string game step-by-step until the string is long enough.

```python
class StringGame:
    def __init__(self):
        """
        Initializes the StringGame with the starting word.
        """
        self.word = "a"

    def _perform_operation(self):
        """
        Performs one operation: generates a new string by shifting characters
        and appends it to the current word.
        This is a private helper method, indicated by the leading underscore.
        """
        new_part = ""
        for char in self.word:
            if char == 'z':
                new_part += 'a'
            else:
                new_part += chr(ord(char) + 1)
        self.word += new_part

    def find_kth_character(self, k: int) -> str:
        """
        Finds the k-th character in the word after enough operations.

        Args:
            k: The 1-indexed position of the character to find.

        Returns:
            The k-th character.
        """
        if k <= 0:
            raise ValueError("k must be a positive integer.")

        # Continue performing operations until the word is long enough
        while len(self.word) < k:
            self._perform_operation()

        # Return the k-th character (adjusting for 0-indexed string)
        return self.word[k - 1]

# Example Usage of OOP Approach 1:
if __name__ == "__main__":
    game_simulator = StringGame()

    # Example 1
    k1 = 5
    result1 = game_simulator.find_kth_character(k1)
    print(f"For k = {k1}, the {k1}-th character is: {result1}") # Expected: b

    # Example 2
    game_simulator_2 = StringGame() # Create a new instance for a fresh start
    k2 = 10
    result2 = game_simulator_2.find_kth_character(k2)
    print(f"For k = {k2}, the {k2}-th character is: {result2}") # Expected: c

    # Example with a small k
    game_simulator_3 = StringGame()
    k3 = 1
    result3 = game_simulator_3.find_kth_character(k3)
    print(f"For k = {k3}, the {k3}-th character is: {result3}") # Expected: a

    # The same instance can be used for multiple calls, but the word will grow
    # If you need a fresh start, create a new instance.
    k4 = 15
    result4 = game_simulator_2.find_kth_character(k4) # Continues from "abbcbccd"
    print(f"For k = {k4}, the {k4}-th character is: {result4}")
    # Let's verify for k=15: "abbcbccd" (length 8)
    # Next operation: new_part from "abbcbccd" is "bccddeef"
    # word becomes "abbcbccdbccddeef" (length 16)
    # k=15 (0-indexed 14) -> 'e' (character at index 14)
```

**OOP Design Choices and Rationale:**

- **`StringGame` Class:** Encapsulates the state (`self.word`) and behavior (`_perform_operation`, `find_kth_character`).
- **`__init__`:** Initializes the game's starting state.
- **`_perform_operation` (Private Helper):** This method is an internal detail of how the `word` is updated. Making it "private" (using the leading underscore) indicates it's not intended for direct external use but rather helps organize the class's internal logic. It modifies `self.word` in place.
- **`find_kth_character` (Public Interface):** This is the main method clients will interact with. It orchestrates the operations until the `k`-th character can be determined.
- **State Management:** The `self.word` attribute holds the current state of the string, which is crucial for subsequent operations.

### OOP Approach 2: Recursive Logic (Optimized for Larger K)

This approach leverages the mathematical pattern observed (recursive nature) and encapsulates it within a class. This is more efficient for very large `k` values, although it's still suitable for the given constraints.

```python
class KthCharacterFinder:
    def __init__(self):
        """
        Initializes the KthCharacterFinder. No initial string is stored
        as the logic is based on direct calculation.
        """
        pass # No state to initialize for this approach

    def _get_shifted_char(self, char: str, shift_amount: int) -> str:
        """
        Helper method to get a character shifted by a certain amount.
        'z' wraps to 'a'.
        """
        start_ord = ord('a')
        current_ord = ord(char)
        shifted_ord = (current_ord - start_ord + shift_amount) % 26 + start_ord
        return chr(shifted_ord)

    def find_kth_character(self, k: int) -> str:
        """
        Finds the k-th character using a recursive, non-simulating approach.

        Args:
            k: The 1-indexed position of the character to find.

        Returns:
            The k-th character.
        """
        if k <= 0:
            raise ValueError("k must be a positive integer.")

        # Adjust k to be 0-indexed for easier calculations
        k_idx = k - 1

        # Determine the smallest power of 2 length that is >= k
        current_length = 1
        while current_length <= k_idx:
            current_length *= 2

        # 'offset' will track how many times the initial 'a' has been shifted
        # due to being in the 'shifted' half of the string
        shift_count = 0

        # Trace back the position and accumulated shifts
        while current_length > 1:
            half_length = current_length // 2

            if k_idx >= half_length:
                # The character is in the second (shifted) half
                k_idx -= half_length # Adjust index to be relative to the shifted part
                shift_count += 1    # One more shift occurred
            # If k_idx < half_length, the character is in the first (original) half
            # No change to k_idx or shift_count needed.

            current_length = half_length

        # At this point, k_idx is 0 (referring to the first character of an 'a'-derived sequence).
        # The character's original value (if it was 'a') has been shifted 'shift_count' times.

        return self._get_shifted_char('a', shift_count)

# Example Usage of OOP Approach 2:
if __name__ == "__main__":
    finder = KthCharacterFinder()

    # Example 1
    k1 = 5
    result1 = finder.find_kth_character(k1)
    print(f"For k = {k1}, the {k1}-th character is: {result1}") # Expected: b

    # Example 2
    k2 = 10
    result2 = finder.find_kth_character(k2)
    print(f"For k = {k2}, the {k2}-th character is: {result2}") # Expected: c

    # Example with a small k
    k3 = 1
    result3 = finder.find_kth_character(k3)
    print(f"For k = {k3}, the {k3}-th character is: {result3}") # Expected: a

    # Example with k that lands on 'z' then wraps to 'a'
    # Consider what happens if k leads to a character far down the alphabet
    # This specific problem only shifts from 'a', so it will always be 'a', 'b', 'c', etc.
    # The wrap-around logic is important if the initial char was different or if the question was modified.
    # For this specific problem, initial_char_code will always be ord('a').

    # Let's try a k that would result in a higher character, e.g., 'd'
    # 'a' -> 'ab' -> 'abbc' -> 'abbcbccd'
    # k=1 (a), k=2 (b), k=3 (b), k=4 (c), k=5 (b), k=6 (c), k=7 (c), k=8 (d)
    k_val_for_d = 8
    result_d = finder.find_kth_character(k_val_for_d)
    print(f"For k = {k_val_for_d}, the {k_val_for_d}-th character is: {result_d}") # Expected: d
```

**OOP Design Choices and Rationale:**

- **`KthCharacterFinder` Class:** This class is designed to find the character without storing the entire string. It's stateless regarding the `word` itself.
- **`__init__`:** Empty, as this approach doesn't maintain a growing `word` state.
- **`_get_shifted_char` (Private Helper):** Encapsulates the logic for character shifting. This makes the `find_kth_character` method cleaner and separates concerns.
- **`find_kth_character` (Public Interface):** This method takes `k` and calculates the result directly using the observed pattern. It doesn't modify any internal state that persists across calls (other than local variables).

### Which OOP approach is better for this specific problem?

For the given constraint `1 <= k <= 500`:

- **OOP Approach 1 (Direct Simulation)** is arguably _more appropriate_ and easier to understand. The $O(k)$ time complexity and $O(k)$ space complexity are well within limits for $k=500$. It directly models the problem statement.
- **OOP Approach 2 (Recursive Logic)** is an _over-optimization_ for this specific constraint but demonstrates a more robust and scalable solution for much larger `k` values (e.g., $k=10^{18}$), where direct simulation would be infeasible due to memory and time constraints. Its $O(\\log k)$ time and $O(1)$ space complexity are superior asymptotically.

Both are valid OOP implementations, but the simulation one aligns more directly with the "game" description for the given `k` range.


In [ ]:
from typing import List

class CharacterFinder:
    def __init__(self):
        self.initial_word = "a"

    def generate_word_until_k(self, k: int) -> str:
        word = self.initial_word

        while len(word) < k:
            next_segment = ""
            for char in word:
                if char == 'z':
                    next_segment += 'a'
                else:
                    next_segment += chr(ord(char) + 1)
            word += next_segment

        return word

    def find_kth_character(self, k: int) -> str:
        full_word = self.generate_word_until_k(k)
        return full_word[k - 1]

# Test cases
if __name__ == "__main__":
    finder = CharacterFinder()
    
    test_cases = [1, 2, 5, 10, 20, 50]
    for k in test_cases:
        result = finder.find_kth_character(k)
        print(f"k = {k}, character = '{result}'")

In [ ]:
class CharacterSimulator:
    def __init__(self):
        self.alphabet_start = ord('a')
        self.alphabet_size = 26

    def find_kth_character(self, k: int) -> str:
        # Adjust k to be 0-indexed
        k -= 1
        current_length = 1

        # Find the smallest power-of-two length that covers index k
        while current_length <= k:
            current_length *= 2

        shift_count = 0  # Number of times the character has been shifted

        # Recursively trace back to the origin character
        while current_length > 1:
            half_length = current_length // 2
            if k >= half_length:
                k -= half_length
                shift_count += 1
            current_length = half_length

        # Compute the final character
        final_char_code = (shift_count % self.alphabet_size) + self.alphabet_start
        return chr(final_char_code)


# ✅ Test cases
if __name__ == "__main__":
    simulator = CharacterSimulator()
    test_values = [1, 5, 10, 26, 50, 100, 500]
    for k in test_values:
        result = simulator.find_kth_character(k)
        print(f"k = {k}, character = '{result}'")

In [ ]:
class Solution:
    def __init__(self):
        self.initial_char = 'a'

    def findKthCharacter(self, k: int) -> str:
        word = self.initial_char

        # Double and transform the string until its length reaches at least k
        while len(word) < k:
            transformed = ""
            for char in word:
                # Shift to the next alphabet letter, wrap 'z' to 'a'
                if char == 'z':
                    transformed += 'a'
                else:
                    transformed += chr(ord(char) + 1)
            word += transformed

        return word[k - 1]  # 1-indexed

# ✅ Test cases
if __name__ == "__main__":
    solution = Solution()
    test_inputs = [1, 2, 5, 10, 20, 50, 100, 512]
    
    print("🔎 Testing findKthCharacter:")
    for k in test_inputs:
        result = solution.findKthCharacter(k)
        print(f"k = {k} ➡ character = '{result}'")